<a href="https://colab.research.google.com/github/sumedh151/pytorch-transformer/blob/main/run_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !python /content/drive/MyDrive/Colab\ Notebooks/attention\ is\ all\ you\ need/main.py

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.

In [2]:
# https://www.youtube.com/watch?v=ISNdQcPhsts

import torch
import torch.nn as nn
import math
# from config import config
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import BPE
from torch.utils.data import Dataset

In [3]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

dataset = load_dataset("cfilt/iitb-english-hindi", split='train')

print(dataset[0])
print(dataset[0]['translation'])
print(dataset[0]['translation']['en'])
print(dataset[0]['translation']['hi'])

if __name__ == '__main__':
    print('hello world')
    # print(config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

{'translation': {'en': 'Give your application an accessibility workout', 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}
{'en': 'Give your application an accessibility workout', 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}
Give your application an accessibility workout
अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
hello world


In [4]:
dataset

Dataset({
    features: ['translation'],
    num_rows: 1659083
})

In [5]:
from datasets import load_dataset
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders, processors

dataset = load_dataset("cfilt/iitb-english-hindi")

# def get_texts(dataset, lang):
def get_texts(dataset):
    for example in dataset['train']['translation']:
        # yield example[lang]
        yield example['en']
        yield example['hi']

tokenizer = Tokenizer(models.BPE())

tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer.decoder = decoders.ByteLevel()
tokenizer.post_processor = processors.ByteLevel()

trainer = trainers.BpeTrainer(special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])

# tokenizer.train_from_iterator(get_texts(dataset, 'en'), trainer=trainer)
# tokenizer.train_from_iterator(get_texts(dataset, 'hi'), trainer=trainer)
tokenizer.train_from_iterator(get_texts(dataset), trainer=trainer)

tokenizer.save("iitb-english-hindi-tokenizer.json")

In [6]:
# Encode some text
encoding = tokenizer.encode("This is a test sentence.")
print(encoding.tokens)

['This', 'Ġis', 'Ġa', 'Ġtest', 'Ġsentence', '.']


In [7]:
tokenizer.get_vocab_size()

30000

In [8]:
# Load the trained tokenizer
tokenizer = Tokenizer.from_file("iitb-english-hindi-tokenizer.json")

# Encode some text
encoding = tokenizer.encode("This is a test sentence.")
print(encoding.tokens)

# Decode back to text
decoded_text = tokenizer.decode(encoding.ids)
print(decoded_text)

['This', 'Ġis', 'Ġa', 'Ġtest', 'Ġsentence', '.']
This is a test sentence.


In [9]:
# Load the trained tokenizer
tokenizer = Tokenizer.from_file("iitb-english-hindi-tokenizer.json")

# Encode some text
encoding = tokenizer.encode("This is a test sentence.")
print(encoding.tokens)

# Decode back to text
decoded_text = tokenizer.decode(encoding.ids)
print(decoded_text)

['This', 'Ġis', 'Ġa', 'Ġtest', 'Ġsentence', '.']
This is a test sentence.


In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [ ]:
class CustomTranslationDataset(Dataset):
    def __init__(self, split='train'):
        self.img_labels = pd.read_csv(annotations_file)

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
CustomTranslationDataset

__main__.CustomTranslationDataset

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [ ]:
class CustomTranslationDataset(Dataset):
    def __init__(self, dataset, tokenizer, split='train'):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.split = split
        self.tokenized_data = self.tokenize_data()

    def tokenize_data(self):
        tokens = []
        for i in self.dataset[self.split]['translation']:
            en_tokens = self.tokenizer.encode(i['en']).ids
            hi_tokens = self.tokenizer.encode(i['hi']).ids
            tokens.append((en_tokens, hi_tokens))
        return tokens

    def __getitem__(self, idx):
        return self.tokenized_data[idx]

    def __len__(self):
        return len(self.tokenized_data)

In [ ]:
ds = CustomTranslationDataset(dataset, tokenizer)

In [ ]:
for i in ds:
    print(i)
    break

([12879, 593, 2865, 388, 20185, 872, 678], [1435, 207, 421, 230, 237, 208, 721, 217, 244, 210, 217, 543, 891, 679, 213, 575, 205, 252, 208, 219, 205, 219, 205, 223, 210, 205, 277, 205, 316, 271, 246])


In [ ]:
for i in ds:
    print(tokenizer.decode(i[0]))
    print(len(tokenizer.decode(i[1])))
    print(len(i[1]))
    break

Give your application an accessibility workout
47
31


In [ ]:
from datasets import load_dataset
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders, processors

dataset = load_dataset("cfilt/iitb-english-hindi")

def get_texts(dataset, lang):
    for example in dataset['train']['translation']:
        yield example[lang]

tokenizer_en = Tokenizer(models.BPE())
tokenizer_en.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer_en.decoder = decoders.ByteLevel()
tokenizer_en.post_processor = processors.ByteLevel()
trainer = trainers.BpeTrainer(special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])
tokenizer_en.train_from_iterator(get_texts(dataset, 'en'), trainer=trainer)

tokenizer_en.save("iitb-english-hindi-tokenizer_en.json")

tokenizer_hi = Tokenizer(models.BPE())
tokenizer_hi.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer_hi.decoder = decoders.ByteLevel()
tokenizer_hi.post_processor = processors.ByteLevel()
trainer = trainers.BpeTrainer(special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])
tokenizer_hi.train_from_iterator(get_texts(dataset, 'hi'), trainer=trainer)

tokenizer_hi.save("iitb-english-hindi-tokenizer_hi.json")

In [ ]:
tokenizer.get_vocab_size()

30000

In [ ]:
max_len=5000
# position = torch.arange(max_len).unsqueeze(1)
position = torch.zeros((max_len,10))
position = torch.zeros((max_len,10)).unsqueeze(1)
position.shape

torch.Size([5000, 1, 10])

In [ ]:
m = nn.Softmax(dim=-1)
input = torch.randn(2, 3)
print(input)
output = m(input)
print(output)

tensor([[-2.4445, -0.3239,  1.1292],
        [-2.4652, -1.6679, -0.8715]])
tensor([[0.0222, 0.1853, 0.7925],
        [0.1228, 0.2726, 0.6046]])


In [129]:
class Encoder(nn.Module):
    def __init__(self, d, dk, dv, h) -> None:
        super(Encoder, self).__init__()
        # self.query_weight = nn.Parameter(torch.randn(d, dk), requires_grad=True)
        # self.key_weight   = nn.Parameter(torch.randn(d, dk), requires_grad=True)
        # self.value_weight = nn.Parameter(torch.randn(d, dv), requires_grad=True)
        self.query_layer = nn.Linear(d,dk)
        self.key_layer = nn.Linear(d,dk)
        self.value_layer = nn.Linear(d,dv)
        self.d = d
        self.dk = dk
        self.dv = dv
        self.h = h
        self.layer_norm1 = nn.LayerNorm(d)
        self.layer_norm2 = nn.LayerNorm(d)
        self.ffo_out = 2048
        self.ffo = nn.Linear(self.h * self.dv , self.d)
        self.ff1 = nn.Linear(self.d , self.ffo_out)
        self.ff2 = nn.Linear(self.ffo_out , self.d)
        self.relu = nn.ReLU()

    def multihead_attention(self, query, key, value):
        # same layers???
        head_outputs = []
        for _ in range(self.h):
            head_output = self.self_attention(query, key, value)
            head_outputs.append(head_output)
        multihead = torch.concat(head_outputs, dim=-1)
        return self.ffo(multihead)

    def self_attention(self, query, key, value):
        qt = torch.bmm(query, torch.transpose(key,1,2))
        attn_weights = nn.Softmax(dim=-1)(qt/np.sqrt(self.dk))
        self_attn = torch.bmm(qt, value)
        return self_attn

    def forward(self,x):
        # query = torch.matmul(x, self.query_weight)
        # key   = torch.matmul(x, self.key_weight)
        # value = torch.matmul(x, self.value_weight)
        query = self.query_layer(x)
        key   = self.key_layer(x)
        value = self.value_layer(x)

        multihead_attn = self.multihead_attention(query, key, value)
        resi1 = multihead_attn + x
        norm1 = self.layer_norm1(resi1)
        feed_forw1 = self.ff1(norm1)
        feed_forw1 = self.relu(feed_forw1)
        feed_forw2 = self.ff2(feed_forw1)
        resi2 = feed_forw2 + norm1
        norm2 = self.layer_norm2(resi2)
        return norm2

In [76]:
from torch import nn
import numpy as np

In [77]:
# output = tokenizer.encode_batch(["Hello, y'all!", "How are you 😁 ?"])
output = tokenizer.encode_batch(["Hello, y'all!", "Hello, y'all!"])
output = [x.ids for x in output]
print(output)
torch.Tensor(output).shape

[[14823, 83, 16, 325, 11, 465, 5], [14823, 83, 16, 325, 11, 465, 5]]


torch.Size([2, 7])

In [78]:
def mask_upper_tri_matrix(matrix):
    upper_triangle_mask = torch.triu(torch.ones_like(matrix, dtype=torch.bool), diagonal=1)
    return matrix.masked_fill(upper_triangle_mask, float('-inf'))

mask_upper_tri_matrix(torch.randn(5, 5))

tensor([[-0.1649,    -inf,    -inf,    -inf,    -inf],
        [ 0.5662,  0.7662,    -inf,    -inf,    -inf],
        [ 0.2261, -0.1001, -1.0840,    -inf,    -inf],
        [ 1.0122, -0.5246, -0.2904,  1.0905,    -inf],
        [ 1.4953, -0.1916,  1.1050, -1.0237, -0.4996]])

In [79]:
class PositionalEncoding(nn.Module):
    def __init__(self) -> None:
        super(PositionalEncoding,self).__init__()

    def forward(self,x):
        return x

In [88]:
class SelfAttention(nn.Module):
    def __init__(self, d, dk, dv, mask):
        super(SelfAttention, self).__init__()
        self.query_layer = nn.Linear(d,dk)
        self.key_layer = nn.Linear(d,dk)
        self.value_layer = nn.Linear(d,dv)
        self.d = d
        self.dk = dk
        self.dv = dv
        self.mask = mask

    def forward(self, x, enc_x=None):
        # print("x shp in enc",x.shape)
        query = self.query_layer(x)
        if enc_x is not None:
            key   = self.key_layer(enc_x)
            value = self.value_layer(enc_x)
        else:
            key   = self.key_layer(x)
            value = self.value_layer(x)
        qt = torch.bmm(query, torch.transpose(key,1,2))
        if self.mask:
            qt = mask_upper_tri_matrix(qt)
        attn_weights = nn.Softmax(dim=-1)(qt/np.sqrt(self.dk))
        self_attn = torch.bmm(qt, value)
        return self_attn

In [89]:
class MultiHeadAttention(nn.Module):
    def __init__(self, h, d, dk, dv, mask=False):
        super(MultiHeadAttention, self).__init__()
        self.h = h
        self.d = d
        self.dk = dk
        self.dv = dv
        self.ffo = nn.Linear(self.h * self.dv , self.d)
        self.mask = mask

    def forward(self,x,enc_x=None):
        head_outputs = []
        for _ in range(self.h):
            head_output = SelfAttention(self.d , self.dk , self.dv , self.mask)(x , enc_x)
            head_outputs.append(head_output)
        multihead = torch.concat(head_outputs, dim=-1)
        return self.ffo(multihead)

In [90]:
class Encoder(nn.Module):
    def __init__(self, d, dk, dv, h) -> None:
        super(Encoder, self).__init__()
        self.d = d
        self.dk = dk
        self.dv = dv
        self.h = h
        self.layer_norm1 = nn.LayerNorm(d)
        self.layer_norm2 = nn.LayerNorm(d)
        self.ffo_out = 2048
        self.ff1 = nn.Linear(self.d , self.ffo_out)
        self.ff2 = nn.Linear(self.ffo_out , self.d)
        self.relu = nn.ReLU()

    def forward(self,x):
        multihead_attn = MultiHeadAttention(self.h, self.d, self.dk, self.dv, mask=False)(x)
        resi1 = multihead_attn + x
        norm1 = self.layer_norm1(resi1)
        feed_forw1 = self.ff1(norm1)
        feed_forw1 = self.relu(feed_forw1)
        feed_forw2 = self.ff2(feed_forw1)
        resi2 = feed_forw2 + norm1
        norm2 = self.layer_norm2(resi2)
        return norm2

In [91]:
class Decoder(nn.Module):
    def __init__(self, d, dk, dv, h) -> None:
        super(Decoder, self).__init__()
        self.h = h
        self.d = d
        self.dk = dk
        self.dv = dv
        self.layer_norm1 = nn.LayerNorm(d)
        self.layer_norm2 = nn.LayerNorm(d)
        self.layer_norm3 = nn.LayerNorm(d)
        self.ffo_out = 2048
        self.ff1 = nn.Linear(self.d , self.ffo_out)
        self.ff2 = nn.Linear(self.ffo_out , self.d)
        self.relu = nn.ReLU()

    def forward(self,x,enc_x):
        multihead_attn1 = MultiHeadAttention(self.h, self.d, self.dk, self.dv, mask=True)(x)
        resi1 = multihead_attn1 + x
        norm1 = self.layer_norm1(resi1)
        multihead_attn2 = MultiHeadAttention(self.h, self.d, self.dk, self.dv, mask=False)(x,enc_x)
        resi2 = multihead_attn2 + norm1
        norm2 = self.layer_norm2(resi2)
        feed_forw1 = self.ff1(norm2)
        feed_forw1 = self.relu(feed_forw1)
        feed_forw2 = self.ff2(feed_forw1)
        resi3 = feed_forw2 + norm2
        norm3 = self.layer_norm2(resi3)
        return norm3

In [107]:
class Transformer(nn.Module):
    def __init__(self, tokenizer, nx=6) -> None:
        super(Transformer, self).__init__()
        self.tokenizer = tokenizer
        self.nx = nx
        self.d = 256
        self.dk = 512
        self.dv = 512
        self.h = 8
        self.inp_embedding = nn.Embedding(tokenizer.get_vocab_size(), self.d, max_norm=True)
        self.out_embedding = nn.Embedding(tokenizer.get_vocab_size(), self.d, max_norm=True)
        self.positional_encoding = PositionalEncoding()
        self.encoder = Encoder(d = self.d, dk = self.dk, dv = self.dv, h = self.h)
        self.decoder = Decoder(d = self.d, dk = self.dk, dv = self.dv, h = self.h)

    def forward(self, input_tokens):
        x = self.inp_embedding(input_tokens)
        x = self.positional_encoding(x)
        for _ in range(self.nx):
            x = self.encoder(x)
        dec_x = self.out_embedding(input_tokens)
        for _ in range(self.nx):
            dec_x = self.decoder(dec_x, x)
        return dec_x

In [108]:
model = Transformer(tokenizer)(torch.LongTensor(output))

In [116]:
output

[[14823, 83, 16, 325, 11, 465, 5], [14823, 83, 16, 325, 11, 465, 5]]

In [117]:
# torch.LongTensor(output)
torch.tensor(output, dtype=torch.int64)

tensor([[14823,    83,    16,   325,    11,   465,     5],
        [14823,    83,    16,   325,    11,   465,     5]])

In [111]:
from torchsummary import summary
summary(model , input_size=(2,7))

AttributeError: 'Tensor' object has no attribute 'apply'

(2, 7)

In [ ]:
torch.LongTensor(output)

tensor([[14823,    83,    16,   325,    11,   465,     5],
        [14823,    83,    16,   325,    11,   465,     5]])

In [ ]:
np.array(output, dtype=np.int32)

array([[14823,    83,    16,   325,    11,   465,     5],
       [14823,    83,    16,   325,    11,   465,     5]], dtype=int32)

In [ ]:
nn.Linear(10,10)

Linear(in_features=10, out_features=10, bias=True)

In [ ]:
l = nn.Linear(10,20)

In [ ]:
x = torch.randn(128, 32,10)
x.shape

torch.Size([128, 32, 10])

In [ ]:
l(x).shape

torch.Size([128, 32, 20])

In [ ]:
l(x).size()

torch.Size([32, 20])

In [103]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 10)  # Adjust 16 * 16 according to your input size

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 64 * 16 * 16)  # Adjust 16 * 16 according to your input size
        x = self.fc1(x)
        return x

In [104]:
def model_summary(model, input_size):
    summary(model, input_size=input_size)

In [105]:
model = MyModel()
input_size = (3, 32, 32)  # Example input size (channels, height, width)
model_summary(model, input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
              ReLU-2           [-1, 64, 32, 32]               0
         MaxPool2d-3           [-1, 64, 16, 16]               0
            Linear-4                   [-1, 10]         163,850
Total params: 165,642
Trainable params: 165,642
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 1.13
Params size (MB): 0.63
Estimated Total Size (MB): 1.77
----------------------------------------------------------------
